# AIS-Matching Workflow

## 1. Get granules info from pyraws

In [ ]:
# See pyraws lib

## 2. Get BBox centers

In [ ]:
#image_to_data_dict_day1

## 3. Start Matching

In [48]:
import pickle
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import numpy as np
from scipy.optimize import linear_sum_assignment
import os


First datetime: 2019-04-15 10:35:05


Select day

In [ ]:
day = 'day8'

# day1 = 2019-02-15
# day2 = 2019-01-21
# day3 = 2019-07-24
# day4 = 2019-07-27
# day5 = 2019-04-20
# day6 = 2019-04-21
# day7 = 2019-06-24
# day8 = 2019-04-15

In [ ]:
# Load the dictionary created with pyraws
with open(f'/Data_large/marine/Datasets/V2RAW/B_02_03_04_08_update/granules/quasiupdate_image_data_{day}.pkl', 'rb') as f:
    image_to_data_dict = pickle.load(f)


# Load the dictionary that contains the BBoxes' center
with open(f'/Data_large/marine/Datasets/V2RAW/B_02_03_04_08_update/dict/{day}_coords_georeferenced.pkl', 'rb') as f:
#with open(f'/Data_large/marine/PythonProjects/db/VENuS/annotations/{day}_coords_georeferenced.pkl', 'rb') as f:
    bbox_centers_dict = pickle.load(f)
    
# Get the first key from the dictionary to set the AIS data
first_key = next(iter(image_to_data_dict))

# Extract the first datetime from the first key's data
first_datetime = image_to_data_dict[first_key][1][0]

# Print the result
print("First datetime:", first_datetime)

In [51]:

# Load AIS data
#df = pd.read_csv('/Data_large/marine/Datasets/DATA_RAW_VESSEL_S2/aisdk-2019-02/aisdk-2019-02-15.csv')
df = pd.read_csv('/Data_large/marine/Datasets/DATA_RAW_VESSEL_S2/aisdk/aisdk_zip/2019-04/aisdk-2019-04-15.csv')
df['Timestamp'] = pd.to_datetime(df['# Timestamp'], dayfirst=True)
df['Time'] = df['Timestamp'].dt.time

In [52]:
def normalize_longitude(lon):
    if lon >= 179 or lon <= -179:
        lon = lon + 180 if lon < 0 else lon - 180
    return lon

def euclidean_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)

def perpendicular_distance(line, point):
    return line.distance(point)

def navigational_status_weight(status):
    if status == 'Engaged in fishing':
        return 0.5  # Prioritize fishing ships by reducing cost
    return 1.0  # Default weight

def process_single_image(selected_key, image_dict, df, bbox_centers_dict):
    matched_entries = []
    unmatched_entries = []
    bbox_gdfs = None

    entry = image_dict[selected_key]
    
    # 1. Extract sensing time and adjust it
    sensing_time = entry[1][0] + pd.Timedelta(seconds=1.8)
    start_time = (sensing_time - pd.Timedelta(seconds=15)).time()
    end_time = (sensing_time + pd.Timedelta(seconds=15)).time()

    # 2. Time filtering: Filter AIS data within the time window
    df_time_filtered = df[(df['Time'] >= start_time) & (df['Time'] <= end_time)].copy()

    # 3. Extract and create a polygon from the image coordinates, reversing them and normalizing longitudes
    coordinates = entry[6]
    reversed_coordinates = [(normalize_longitude(lon), lat) for lat, lon in coordinates]
    polygon = Polygon(reversed_coordinates)

    # 4. Spatial filtering: Filter AIS data within the polygon
    df_time_filtered['geometry'] = [Point(normalize_longitude(lon), lat) for lon, lat in zip(df_time_filtered['Longitude'], df_time_filtered['Latitude'])]
    geo_df = gpd.GeoDataFrame(df_time_filtered, geometry='geometry')
    inside_polygon = geo_df.within(polygon)
    geo_df_within_polygon = geo_df.loc[inside_polygon].copy()

    if not geo_df_within_polygon.empty:
        mmsi_to_coords = {}

        # Process each unique MMSI: Get the nearest two timestamps for each MMSI
        for mmsi in geo_df_within_polygon['MMSI'].unique():
            mmsi_filtered = geo_df_within_polygon.loc[geo_df_within_polygon['MMSI'] == mmsi].copy()
            mmsi_filtered['time_diff'] = (mmsi_filtered['Timestamp'] - sensing_time).abs()
            sorted_entries = mmsi_filtered.sort_values('time_diff').iloc[:2]
            coordinates_list = sorted_entries[['Latitude', 'Longitude']].values.tolist()
            heading = sorted_entries.iloc[0]['Heading']
            navigational_status = sorted_entries.iloc[0]['Navigational status']
            mmsi_to_coords[mmsi] = {
                'coords': coordinates_list,
                'heading': heading,
                'navigational_status': navigational_status,
                'info': sorted_entries[['Timestamp', 'Heading', 'Navigational status', 'Name', 'Ship type', 'Cargo type']].to_dict(orient='records')
            }

        # Matching algorithm with Hungarian Algorithm
        if selected_key in bbox_centers_dict:
            bbox_centers = bbox_centers_dict[selected_key]
            used_mmsi = set()
            bbox_centers_list = []  # List to store bbox centers for each image

            bbox_points = [Point(normalize_longitude(center[1]), center[0]) for _, _, _, center in bbox_centers]

            cost_matrix = np.zeros((len(bbox_points), len(mmsi_to_coords)))

            for i, bbox_point in enumerate(bbox_points):
                for j, (mmsi, data) in enumerate(mmsi_to_coords.items()):
                    coords = data['coords']
                    heading = data['heading']
                    navigational_status = data['navigational_status']
                    
                    if len(coords) == 2:
                        line = LineString([(normalize_longitude(coords[0][1]), coords[0][0]), (normalize_longitude(coords[1][1]), coords[1][0])])
                        perp_distance = perpendicular_distance(line, bbox_point)
                    else:
                        perp_distance = euclidean_distance(bbox_point.y, bbox_point.x, coords[0][0], normalize_longitude(coords[0][1]))
                    
                    eucl_distance = euclidean_distance(bbox_point.y, bbox_point.x, coords[0][0], normalize_longitude(coords[0][1]))
                    status_weight = navigational_status_weight(navigational_status)
                    
                    cost_matrix[i, j] = status_weight * (perp_distance + eucl_distance)

            # Handle invalid numeric entries in the cost matrix
            cost_matrix[np.isnan(cost_matrix)] = float('inf')
            cost_matrix[np.isinf(cost_matrix)] = float('inf')

            if np.all(cost_matrix == float('inf')) or cost_matrix.size == 0:
                print(f"No feasible matches for {selected_key}. All distances are infinite or cost matrix is empty.")
                unmatched_entries = [{'bbox_center': bbox_centers[i][3]} for i in range(len(bbox_centers))]
            else:
                try:
                    row_ind, col_ind = linear_sum_assignment(cost_matrix)
                except ValueError as e:
                    print(f"ValueError for {selected_key}: {e}")
                    unmatched_entries = [{'bbox_center': bbox_centers[i][3]} for i in range(len(bbox_centers))]
                    row_ind, col_ind = [], []

                for i, bbox_index in enumerate(row_ind):
                    mmsi_index = col_ind[i]
                    if cost_matrix[bbox_index, mmsi_index] != float('inf'):
                        mmsi = list(mmsi_to_coords.keys())[mmsi_index]
                        data = mmsi_to_coords[mmsi]
                        matched_entries.append({
                            'mmsi': mmsi,
                            'coordinates': [data['coords'][0]],  # Use only the nearest AIS coordinate
                            'bbox_center': bbox_centers[bbox_index][3],
                            'distance': cost_matrix[bbox_index, mmsi_index],
                            'info': data['info']
                        })
                        used_mmsi.add(mmsi)

                # Add unmatched MMSIs
                for mmsi, data in mmsi_to_coords.items():
                    if mmsi not in used_mmsi:
                        unmatched_entries.append({
                            'mmsi': mmsi,
                            'coordinates': data['coords'],
                            'info': data['info']
                        })

                # Store the bbox centers as a list of dictionaries
                bbox_centers_list = [{'geometry': Point(normalize_longitude(center[1]), center[0]), 'lat': center[0], 'lon': center[1]} for _, _, _, center in bbox_centers]
                
                # Convert list to GeoDataFrame
                bbox_gdfs = gpd.GeoDataFrame(bbox_centers_list)

    # Save the AIS GeoDataFrame after time and spatial filtering:
        # .shp: The main file that contains the geometric data (points, lines, polygons).
        # .shx: The shape index file, which stores the index of the geometry.
        # .dbf: The attribute data file, which stores the attribute data for each shape in a tabular format.
        # .prj: The projection file, which contains information about the coordinate system and projection of the geometric data.
    if not geo_df_within_polygon.empty:
        geo_df_within_polygon = geo_df_within_polygon.copy()
        geo_df_within_polygon['Timestamp'] = geo_df_within_polygon['Timestamp'].astype(str)  # Convert datetime to string
        geo_df_within_polygon['Time'] = geo_df_within_polygon['Time'].astype(str)  # Convert time to string
        
        #ais_gdf_dir = '/Data_large/marine/prova/VENuS_S2/S2/AIS/results2/gdf'
        ais_gdf_dir ='/Data_large/marine/Datasets/V2RAW/B_02_03_04_08_update/prova/gdf'
        # Construct the output directory path
        ais_gdf_output_dir = os.path.join(ais_gdf_dir, selected_key)
        # Create the directory if it does not exist
        os.makedirs(ais_gdf_output_dir, exist_ok=True)
        
        output_file = os.path.join(ais_gdf_output_dir, f'AIS_filtered_{selected_key}.shp')
        geo_df_within_polygon.to_file(output_file)
        
        #geo_df_within_polygon.to_file(f'/Data_large/marine/PythonProjects/db/VENuS/results/S2_AIS_matching/gdf/{selected_key}/AIS_filtered_{selected_key}.shp')

    return matched_entries, unmatched_entries, bbox_gdfs

def process_all_images(image_dict, df, bbox_centers_dict):
    all_matched_entries = {}
    all_unmatched_entries = {}
    all_bbox_gdfs = {}

    for selected_key in image_dict.keys():
        matched_entries, unmatched_entries, bbox_gdfs = process_single_image(selected_key, image_dict, df, bbox_centers_dict)
        all_matched_entries[selected_key] = matched_entries
        all_unmatched_entries[selected_key] = unmatched_entries
        all_bbox_gdfs[selected_key] = bbox_gdfs

    return all_matched_entries, all_unmatched_entries, all_bbox_gdfs


In [ ]:
image_dict = image_to_data_dict

# ## SINGLE IMAGE PROCESS
# selected_key = 'day1_g_21_coreg.tif'
# matched_entries, unmatched_entries, bbox_gdfs = process_single_image(selected_key, image_dict, df, bbox_centers_dict)

# ALL IMAGES PROCESS
all_matched_data, all_unmatched_data, all_bbox_gdfs = process_all_images(image_to_data_dict, df, bbox_centers_dict)

In [55]:

# Save the matched data to a pickle file for further analysis
with open(f'/Data_large/marine/Datasets/V2RAW/B_02_03_04_08_update/prova/results/{day}_matched_hung.pkl', 'wb') as f:
    pickle.dump(all_matched_data, f)

In [ ]:
import pickle
# Read and check
with open(f'/Data_large/marine/Datasets/V2RAW/B_02_03_04_08_update/prova/results/day8_matched_hung.pkl', 'rb') as f:
    match_read=pickle.load(f)
    
match_read

In [24]:

# Save the matched data to a pickle file for further analysis
with open(f'/Data_large/marine/prova/VENuS_S2/S2/AIS/results2/{day}_matched_hung.pkl', 'wb') as f:
    pickle.dump(all_matched_data, f)

# Save the unmatched data to a pickle file for further analysis
with open(f'/Data_large/marine/prova/VENuS_S2/S2/AIS/results2/{day}_unmatched_hung.pkl', 'wb') as f:
    pickle.dump(all_unmatched_data, f)

# Optionally, save bbox GeoDataFrames to shapefiles
for key, gdf in all_bbox_gdfs.items():
    if gdf is not None and not gdf.empty:        
        gdf.to_file(f'/Data_large/marine/prova/VENuS_S2/S2/AIS/results2/bbox/bbox_centers_{key}.shp')